## Import the necessary libraries

In [2]:
import io
import zipfile
import requests
import frontmatter

## Complete function

In [3]:
def read_repo_data(repo_owner, repo_name):
    """
    Download and parse all markdown files from a GitHub repository.
    
    Args:
        repo_owner: GitHub username or organization
        repo_name: Repository name
    
    Returns:
        List of dictionaries containing file content and metadata
    """
    prefix = 'https://codeload.github.com' 
    url = f'{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/main'
    resp = requests.get(url)
    
    if resp.status_code != 200:
        raise Exception(f"Failed to download repository: {resp.status_code}")

    repository_data = []
    zf = zipfile.ZipFile(io.BytesIO(resp.content))
    
    for file_info in zf.infolist():
        filename = file_info.filename
        filename_lower = filename.lower()

        if not (filename_lower.endswith('.md') 
            or filename_lower.endswith('.mdx')):
            continue
    
        try:
            with zf.open(file_info) as f_in:
                content = f_in.read().decode('utf-8', errors='ignore')
                post = frontmatter.loads(content)
                data = post.to_dict()
                data['filename'] = filename
                repository_data.append(data)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue
    
    zf.close()
    return repository_data


In [4]:
qgis_web = read_repo_data('qgis', 'QGIS-Website')

Error processing QGIS-Website-main/content/project/case-studies/italy_cesena.md: while parsing a quoted scalar
  in "<unicode string>", line 2, column 8
found unknown escape character
  in "<unicode string>", line 2, column 36


In [5]:
print(f"QGIS Web documents: {len(qgis_web)}")

QGIS Web documents: 880
